<a href="https://colab.research.google.com/github/DerErnest2/axa-citibike-challenge/blob/main/notebooks/01_data_ingestion_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
DRIVE_PATH = '/content/drive/MyDrive/axa-challenge/data/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
# !git config --global user.name "DerErnest2"
# !git config --global user.email "tstuerzebecher@gmail.com"
# !git clone "https://github.com/DerErnest2/axa-citibike-challenge"

Cloning into 'axa-citibike-challenge'...
remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 13 (delta 2), reused 7 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (13/13), 4.91 KiB | 4.91 MiB/s, done.
Resolving deltas: 100% (2/2), done.


In [ ]:
%cd axa-citibike-challenge

/content/axa-citibike-challenge/axa-citibike-challenge


In [ ]:
!pip install -r requirements.txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.6 MB/s eta 0:00:00
